# Protein feature extraction pipeline

This notebook will contain the pipeline for extracting features from protein sequences. It will be used as a way to show the output without needing to run the `pipeline.py` file locally.

In [1]:
import pyarrow as pa
import pandas as pd
import os
import glob
import logging
# from fondant.pipeline import Pipeline
from fondant.dataset import Dataset
from fondant.dataset.runner import DockerRunner

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

from config import MOCK_DATA_PATH_FONDANT

# check if the manifest file is removed.
REMOVED_MANIFEST = False

# check if the output folder exists
OUTPUT_FOLDER = None

## Generate Mock data

In [2]:
!python utils/generate_mock_data.py

In [3]:
# show content of the mock data
import pandas as pd
mock_df = pd.read_parquet("." + MOCK_DATA_PATH_FONDANT)  # dot added to make it relative to the current directory
mock_df

,sequence,name
0,MNQRGMPIQSLVTNVKINRLEENDCIHTRHRVRPGRTDGKNLHAMM...,Seq1
1,MAGLKPEVPLHDGINKFGKSDFAGQEGPKIVTTTDKALLVANGALK...,Seq2
2,MVDLKKELKNFVDSDFPGSPKQEAQGIDVRILLSFNNAAFREALII...,Seq3
3,MELILAKARLEFECDWGLLMLEPCVPPTKIFADRNYAVGVMFESDK...,Seq4
4,MRVLCDGSTGYACAKNTRIRFREKVASVLAKIQGYEQTFPHHMPNM...,Seq5
5,MRVLCDGSTGYACAKNTRIRFREKVASVLAKIQGYEQTFPHHMPNM...,Seq6
6,MRVLCDGSTGYACAKNTRIRFREKVASVLAKIQGYEQTFPHHMPNM...,Seq7
7,MRVLCDGSTGYACAKNTRIRFREKVASVLAKIQGYEQTFPHHMPNM...,Seq8


## Loading the dataset

In [4]:
# Create a new pipeline

BASE_PATH = "."
DATASET_NAME = "feature_extraction_pipeline"

dataset = Dataset.create(
    "load_from_parquet",
    arguments={
        "dataset_uri": MOCK_DATA_PATH_FONDANT,
    },
    produces={
        "sequence": pa.string()
    },
    dataset_name=DATASET_NAME
)


[2024-09-05 11:55:37,310 | fondant.dataset.dataset | INFO] The consumes section of the component spec is not defined. Can not infer consumes of the OperationSpec. Please define a consumes section in the dataset interface. 


## Creating the pipeline

## Components

---

### generate_protein_sequence_checksum_component

This component generates a checksum for the protein sequence.

---

### biopython_component

Extracts features from the protein sequence using Biopython.

---

### iFeatureOmega_component

Extracts features from the protein sequence using the [iFeatureOmega-CLI GitHub repo](https://github.com/Superzchen/iFeatureOmega-CLI). Arguments are used to specify the type of features to extract.

---

### filter_pdb_component

Filters PDB files that are already predicted to avoid redundant predictions. Arguments need to be specified before running the pipeline:
```json
"storage_type": "local",
"pdb_path": "/data/<your-pdb-folder-path>",
"bucket_name": "your-bucket-name",
"project_id": "your-project-id",
"google_cloud_credentials_path": "/data/<your-credentials>.json"
```

If only using local, keep bucket_name, project_id, and google_cloud_credentials_path as empty strings. Using remote requires a Google Cloud Storage bucket with credentials and a project ID.

---

### predict_protein_3D_structure_component

Predicts the 3D structure of the protein using ESMFold. This component requires a `.env` file with the following variables:
```env
HF_API_KEY=""
HF_ENDPOINT_URL=""
```

---

### store_pdb_component

Stores the PDB files in the provided storage_type. Arguments need to be specified before running the pipeline:
```json
"storage_type": "local",
"pdb_path": "/data/<your-pdb-folder-path>",
"bucket_name": "your-bucket-name",
"project_id": "your-project-id",
"google_cloud_credentials_path": "/data/<your-credentials>.json"
```

If only using local, keep bucket_name, project_id, and google_cloud_credentials_path as empty strings. Using remote requires a Google Cloud Storage bucket with credentials and a project ID.

---

### msa_component

Generates the multiple sequence alignment for the protein sequence using [Clustal Omega](http://www.clustal.org/omega/). It's recommended to use a smaller number of sequences or none at all due to potential time consumption.

---

### unikp_component

Uses the UniKP endpoint on HuggingFace to predict the kinetic parameters of a protein sequence and substrate (SMILES) combination. See README for the description of the contents of this file.

```yaml
"protein_smiles_path": "/data/<path_protein_smiles>"
```

---

### peptide_component

Calculates the features from the protein sequence using the `peptides` package.

---

### deepTMpred_component

Predicts the transmembrane regions of the protein sequence using the [DeepTMpred GitHub repository](https://github.com/ISYSLAB-HUST/DeepTMpred)

In [5]:
dataset = dataset.apply(
    "./components/biopython_component",
    cache=False
).apply(
    "./components/generate_protein_sequence_checksum_component",
    cache=False
).apply(
    "./components/iFeatureOmega_component",
    # currently forcing the number of rows to 5, but there needs to be a better way to do this, see readme for more info
    input_partition_rows=5,
    arguments={
        "descriptors": ["AAC", "CTDC", "CTDT"]
    }
).apply(
    "./components/filter_pdb_component",
    arguments={
        "method": "local",
        "local_pdb_path": "/data/pdb_files",
        "bucket_name": "",
        "project_id": "",
        "google_cloud_credentials_path": ""
    }
).apply(
    "./components/predict_protein_3D_structure_component",
).apply(
    "./components/store_pdb_component",
    arguments={
        "method": "local",
        "local_pdb_path": "/data/pdb_files/",
        "bucket_name": "elated-chassis-400207_dbtl_pipeline_outputs",
        "project_id": "elated-chassis-400207",
        "google_cloud_credentials_path": "/data/google_cloud_credentials.json"
    }
).apply(
    "./components/msa_component",
    input_partition_rows='10000'
# ).apply(
#     "./components/pdb_features_component"
# ).apply(
#     "./components/unikp_component",
#     arguments={
#         "target_molecule_smiles": "/data/target_molecule_smiles.json",
#     },
# ).apply(
#     "./components/peptide_features_component"
# ).apply(
#     "./components/DeepTMpred_component"
)

[2024-09-05 11:55:37,360 | fondant.dataset.dataset | WARNING] Component `iFeatureOmega component` has an image tag set to latest. Caching for the component will be disabled to prevent unpredictable behavior due to images updates
[2024-09-05 11:55:37,366 | fondant.dataset.dataset | WARNING] Component `Filter PDB Component` has an image tag set to latest. Caching for the component will be disabled to prevent unpredictable behavior due to images updates
[2024-09-05 11:55:37,372 | fondant.dataset.dataset | WARNING] Component `Predict Protein 3D Structure Component` has an image tag set to latest. Caching for the component will be disabled to prevent unpredictable behavior due to images updates
[2024-09-05 11:55:37,381 | fondant.dataset.dataset | WARNING] Component `Store PDB Component` has an image tag set to latest. Caching for the component will be disabled to prevent unpredictable behavior due to images updates
[2024-09-05 11:55:37,386 | fondant.dataset.dataset | WARNING] Component `MSA

## Run the pipeline

The `pipeline.py` file needs to be run using the command line. The following command will run the pipeline:

```bash
fondant < full_path_to_pipeline.py >\data:/data
```

In [6]:
# import shutil

# remove the most recent output folder if the manifest file is removed
# without a manifest file in the most recent output folder, the pipeline cannot be run
# if OUTPUT_FOLDER and REMOVED_MANIFEST:
# 	shutil.rmtree(OUTPUT_FOLDER)
# 	# remove cache
# 	shutil.rmtree(os.path.join(BASE_PATH, PIPELINE_NAME, "cache"))

# get current full path to the project
mounted_data = os.path.join(os.path.abspath("data"), ":/data")

DockerRunner().run(dataset=dataset, working_directory=BASE_PATH, extra_volumes=mounted_data)

[2024-09-05 11:55:37,458 | root | INFO] Found reference to un-compiled workflow... compiling
[2024-09-05 11:55:37,459 | fondant.dataset.compiler | INFO] Base path found on local system, setting up . as mount volume
[2024-09-05 11:55:37,459 | fondant.dataset.dataset | INFO] Sorting workflow graph topologically.
[2024-09-05 11:55:37,479 | fondant.dataset.dataset | INFO] All workflow component specifications match.
[2024-09-05 11:55:37,480 | fondant.dataset.compiler | INFO] Compiling service for load_from_parquet
[2024-09-05 11:55:37,481 | fondant.dataset.compiler | INFO] Compiling service for biopython_component
[2024-09-05 11:55:37,482 | fondant.dataset.compiler | INFO] Found Dockerfile for biopython_component, adding build step.
[2024-09-05 11:55:37,482 | fondant.dataset.compiler | INFO] Compiling service for generate_protein_sequence_checksum_component
[2024-09-05 11:55:37,483 | fondant.dataset.compiler | INFO] Found Dockerfile for generate_protein_sequence_checksum_component, adding 

Docker version:
(26, 1, 4)
Starting workflow run...


 load_from_parquet Pulling 
 load_from_parquet Pulled 


#0 building with "desktop-linux" instance using docker driver

#1 [biopython_component internal] load build definition from Dockerfile
#1 transferring dockerfile: 466B done
#1 DONE 0.0s

#2 [biopython_component internal] load metadata for docker.io/library/python:3.10-slim
#2 DONE 0.8s

#3 [biopython_component internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [biopython_component 1/6] FROM docker.io/library/python:3.10-slim@sha256:235703999ce36b498d490b534e781664d72cdec9c2f0adb97582467e55fad8e2
#4 DONE 0.0s

#5 [biopython_component internal] load build context
#5 transferring context: 3.63kB done
#5 DONE 0.0s

#6 [biopython_component 5/6] WORKDIR /component/src
#6 CACHED

#7 [biopython_component 3/6] COPY requirements.txt ./
#7 CACHED

#8 [biopython_component 4/6] RUN pip3 install --no-cache-dir -r requirements.txt
#8 CACHED

#9 [biopython_component 2/6] RUN apt-get update &&     apt-get upgrade -y &&     apt-get install git -y
#9 CACHED

#10 [biopython_com

 Container feature_extraction_pipeline-load_from_parquet-1  Recreate
 Container feature_extraction_pipeline-load_from_parquet-1  Recreated
 Container feature_extraction_pipeline-biopython_component-1  Recreate
 Container feature_extraction_pipeline-biopython_component-1  Recreated
 Container feature_extraction_pipeline-generate_protein_sequence_checksum_component-1  Recreate
 Container feature_extraction_pipeline-generate_protein_sequence_checksum_component-1  Recreated
 Container feature_extraction_pipeline-ifeatureomega_component-1  Recreate
 Container feature_extraction_pipeline-ifeatureomega_component-1  Recreated
 Container feature_extraction_pipeline-filter_pdb_component-1  Recreate
 Container feature_extraction_pipeline-filter_pdb_component-1  Recreated
 Container feature_extraction_pipeline-predict_protein_3d_structure_component-1  Recreate
 Container feature_extraction_pipeline-predict_protein_3d_structure_component-1  Recreated
 Container feature_extraction_pipeline-store_pdb

...

predict_protein_3d_structure_component-1 exited with code 0


store_pdb_component-1                           | [2024-09-05 09:56:16,392 | fondant.cli | INFO] Component `StorePDBComponent` found in module main
store_pdb_component-1                           | [2024-09-05 09:56:16,400 | fondant.component.executor | INFO] Caching disabled for the component
store_pdb_component-1                           | [2024-09-05 09:56:16,401 | root | INFO] Executing component
store_pdb_component-1                           | [2024-09-05 09:56:16,712 | distributed.http.proxy | INFO] To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
store_pdb_component-1                           | [2024-09-05 09:56:16,739 | distributed.scheduler | INFO] State start
store_pdb_component-1                           | [2024-09-05 09:56:16,742 | distributed.scheduler | INFO]   Scheduler at:     tcp://127.0.0.1:35799
store_pdb_component-1                           | [2024-09-05 09:56:16,743 | distributed.schedul

store_pdb_component-1 exited with code 0


msa_component-1                                 | [2024-09-05 09:56:22,091 | fondant.cli | INFO] Component `MSAComponent` found in module main
msa_component-1                                 | [2024-09-05 09:56:22,100 | fondant.component.executor | INFO] Caching disabled for the component
msa_component-1                                 | [2024-09-05 09:56:22,100 | root | INFO] Executing component
msa_component-1                                 | [2024-09-05 09:56:22,472 | distributed.http.proxy | INFO] To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
msa_component-1                                 | [2024-09-05 09:56:22,497 | distributed.scheduler | INFO] State start
msa_component-1                                 | [2024-09-05 09:56:22,501 | distributed.scheduler | INFO]   Scheduler at:     tcp://127.0.0.1:39887
msa_component-1                                 | [2024-09-05 09:56:22,501 | distributed.scheduler | 

msa_component-1 exited with code 1
Finished workflow run.


## Results

The following results have been taken from the output of the pipeline, which is stored in the `.fondant` directory. This directory contains the output of each component, together with the cache of the previous run. Currently, the pipeline doesn't implement the `write_to_file` component, so the results will be taken individually from the output of each component.

In [7]:
# find the most recent output folder
# get the most recent folder in the folder named: BASE_PATH + PIPELINE_NAME + PIPELINE_NAME-<timestamp>
matching_folders = glob.glob(f"{BASE_PATH}/{DATASET_NAME}/{DATASET_NAME}-*")

if matching_folders:
    last_folder = max(matching_folders, key=os.path.getctime)

logging.info(f"Last folder: {last_folder}")


[2024-09-05 11:56:25,308 | root | INFO] Last folder: ./feature_extraction_pipeline/feature_extraction_pipeline-20240905115537


In [8]:
from pathlib import Path

def merge_parquet_folders(folder_path):
    df_list = []

    for folder in Path(folder_path).iterdir():
        if folder.is_dir():
            logging.info(f"Reading parquet partitions from: {folder}")
            parquet_files = list(folder.glob("*.parquet"))
            logging.info(f"Found {len(parquet_files)} parquet files")
            dfs = [pd.read_parquet(file) for file in parquet_files]
            dfs = [x for x in dfs if not x.empty]
            if len(dfs) == 0:
                continue
            df = pd.concat(dfs)
            df_list.append(df)

    return df_list

In [9]:
dataframe_list = merge_parquet_folders(last_folder)


df_final = pd.concat(dataframe_list, axis=1)
df_final = df_final.loc[:,~df_final.columns.duplicated()]




[2024-09-05 11:56:25,333 | root | INFO] Reading parquet partitions from: feature_extraction_pipeline/feature_extraction_pipeline-20240905115537/filter_pdb_component
[2024-09-05 11:56:25,335 | root | INFO] Found 8 parquet files
[2024-09-05 11:56:25,374 | root | INFO] Reading parquet partitions from: feature_extraction_pipeline/feature_extraction_pipeline-20240905115537/predict_protein_3d_structure_component
[2024-09-05 11:56:25,375 | root | INFO] Found 8 parquet files
[2024-09-05 11:56:25,408 | root | INFO] Reading parquet partitions from: feature_extraction_pipeline/feature_extraction_pipeline-20240905115537/biopython_component
[2024-09-05 11:56:25,409 | root | INFO] Found 8 parquet files
[2024-09-05 11:56:25,440 | root | INFO] Reading parquet partitions from: feature_extraction_pipeline/feature_extraction_pipeline-20240905115537/load_from_parquet
[2024-09-05 11:56:25,441 | root | INFO] Found 0 parquet files
[2024-09-05 11:56:25,441 | root | INFO] Reading parquet partitions from: featu

In [10]:
df_final[["sequence", "pdb_string", "msa_sequence"]].to_json("test_json.json", orient="records")

KeyError: "['msa_sequence'] not in index"

In [ ]:
# filtering out columns that are not properly stored in a csv
columns_to_remove = ['pdb_string']
df_final = df_final.drop(columns=columns_to_remove)

# write to file
df_final.to_csv(f"{last_folder}/final_output.csv", index=False)